In [24]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus, LpStatusOptimal, LpInteger
import numpy as np
import pandas as pd
import train_co2_emissions
import fuel_consumption_calc
import function_data

In [25]:
df_train = pd.read_csv('train_lines.csv')
train_cities = pd.read_csv('train_cities.csv')
df_plane = pd.read_csv('data/Flights_20191201_20191231.csv')

In [26]:
df_trains_arcs = train_co2_emissions.treatment(df_train)
df_trains_arcs.head()

Le dataframe a été mis à jour avec la colonne Emissions.


,ville_dep,ville_arr,iuc_dep,iuc_arr,duree_min,Emissions_Train_kgCO2
0,Bruxelles,Amsterdam,8814001,8400058,110,5.568750
1,Bruxelles,Anvers,8814001,8821006,34,1.586667
2,Anvers,Rotterdam,8821006,8400530,36,1.822500
3,Bruxelles,Liège,8814001,8841006,65,3.033333
4,Liège,Cologne,8841006,8000207,89,5.377083


In [48]:
def create_arcs(df, mode):
    arcs = {}
    for row in df.itertuples(index=False):
        if mode == "train" :
            key_1 = (row.ville_dep, row.ville_arr, mode)
            key_2 = (row.ville_arr, row.ville_dep, mode)
            value = (row.Emissions_Train_kgCO2, row.duree_min)
            # Si l'arc existe déjà, on peut choisir de garder le moins émetteur ou le plus rapide (dans le cas ou une ville a deux aéroports ou deux gares)
            for key in [key_1, key_2]:
                if key in arcs:
                    existing_emissions, existing_time = arcs[key]
                    if row.Emissions_Train_kgCO2 < existing_emissions:
                        arcs[key] = (row.Emissions_Train_kgCO2, row.duree_min)
                    elif row.Emissions_Train_kgCO2 == existing_emissions and row.duree_min < existing_time:
                        arcs[key] = (row.Emissions_Train_kgCO2, row.duree_min)
                else:
                    arcs[key] = value
        elif mode == "avion":
            key = (row.Ville_DEP, row.Ville_DES, mode)
            value = (row.mean_emissions, row.mean_time)
            if key in arcs:
                existing_emissions, existing_time = arcs[key]
                if row.mean_emissions < existing_emissions:
                    arcs[key] = value
                elif row.mean_emissions == existing_emissions and row.mean_time < existing_time:
                    arcs[key] = value
            else:
                arcs[key] = value
    return arcs

In [54]:
def complete_graph(df_plane, plane_arcs):
    count = 0
    for row in df_plane.itertuples(index=False):
        key_1 = (row.Ville_DEP, row.Ville_DES, "avion")
        key_2 = (row.Ville_DES, row.Ville_DEP, "avion")
        if (key_1 in plane_arcs) and (key_2 not in plane_arcs):
            plane_arcs[key_2] = plane_arcs[key_1]
            count+=1
        elif (key_2 in plane_arcs) and (key_1 not in plane_arcs):
            plane_arcs[key_1] = plane_arcs[key_2]
            count+=1
    print(f"{count} return arcs completed")
    return plane_arcs


In [50]:
train_arcs = create_arcs(df_trains_arcs, "train")
print(train_arcs)

{('Bruxelles', 'Amsterdam', 'train'): (5.56875, 110), ('Amsterdam', 'Bruxelles', 'train'): (5.56875, 110), ('Bruxelles', 'Anvers', 'train'): (1.5866666666666667, 34), ('Anvers', 'Bruxelles', 'train'): (1.5866666666666667, 34), ('Anvers', 'Rotterdam', 'train'): (1.8225, 36), ('Rotterdam', 'Anvers', 'train'): (1.8225, 36), ('Bruxelles', 'Liège', 'train'): (3.033333333333333, 65), ('Liège', 'Bruxelles', 'train'): (3.033333333333333, 65), ('Liège', 'Cologne', 'train'): (5.377083333333334, 89), ('Cologne', 'Liège', 'train'): (5.377083333333334, 89), ('Lille', 'Bruxelles', 'train'): (1.1484375, 35), ('Bruxelles', 'Lille', 'train'): (1.1484375, 35), ('Lyon', 'Marseille', 'train'): (0.875, 100), ('Marseille', 'Lyon', 'train'): (0.875, 100), ('Madrid', 'Barcelone', 'train'): (2.964, 156), ('Barcelone', 'Madrid', 'train'): (2.964, 156), ('Madrid', 'Seville', 'train'): (2.9069999999999996, 153), ('Seville', 'Madrid', 'train'): (2.9069999999999996, 153), ('Seville', 'Cadix', 'train'): (1.596, 84),

In [51]:
df_planes_arcs = function_data.treatment(df_plane)
df_planes_arcs.head()

Nombre de vols europe : 530504
Nombre de vols europe filtrés : 411148


,ADEP,ADES,mean_emissions,mean_time,num_flights,Ville_DEP,Ville_DES
1652,LTFJ,LTBJ,2618.039751,59.928616,742,Istanbul,Izmir
1745,LTBJ,LTFJ,2682.141134,54.847230,734,Izmir,Istanbul
1711,LTAC,LTFJ,2571.554358,48.169105,704,Ankara,Istanbul
1642,LTFJ,LTAC,2471.774927,52.996930,695,Istanbul,Ankara
683,LEMD,LEBL,3102.965256,67.040615,650,Madrid,Barcelone


In [52]:
plane_arcs = create_arcs(df_planes_arcs, "avion")
print(plane_arcs)

{('Istanbul', 'Izmir', 'avion'): (2618.0397509479253, 59.92861635220126), ('Izmir', 'Istanbul', 'avion'): (2682.141134488489, 54.847229791099004), ('Ankara', 'Istanbul', 'avion'): (2571.5543575920087, 48.16910511363637), ('Istanbul', 'Ankara', 'avion'): (2471.7749270351173, 52.99693045563549), ('Madrid', 'Barcelone', 'avion'): (3102.9652564629973, 67.04061538461539), ('Barcelone', 'Madrid', 'avion'): (3019.863101323577, 72.00526315789475), ('Trondheim', 'Oslo', 'avion'): (1124.79009033914, 68.64521276595744), ('Oslo', 'Trondheim', 'avion'): (1166.3191420181638, 73.35514184397162), ('Amsterdam', 'Londres', 'avion'): (1570.3234874094119, 56.29036036036036), ('Londres', 'Amsterdam', 'avion'): (1611.5808663578828, 50.44108108108108), ('Barcelone', 'Palma de Majorque', 'avion'): (1875.0021478076717, 45.485358796296296), ('Bergen', 'Oslo', 'avion'): (822.5813668700935, 48.92169312169312), ('Palma de Majorque', 'Barcelone', 'avion'): (1953.1258548627366, 43.42555168408827), ('Munich', 'Hambou

In [55]:
final_plane_arcs = complete_graph(df_planes_arcs, plane_arcs)
print(final_plane_arcs)

0 return arcs completed
{('Istanbul', 'Izmir', 'avion'): (2618.0397509479253, 59.92861635220126), ('Izmir', 'Istanbul', 'avion'): (2682.141134488489, 54.847229791099004), ('Ankara', 'Istanbul', 'avion'): (2571.5543575920087, 48.16910511363637), ('Istanbul', 'Ankara', 'avion'): (2471.7749270351173, 52.99693045563549), ('Madrid', 'Barcelone', 'avion'): (3102.9652564629973, 67.04061538461539), ('Barcelone', 'Madrid', 'avion'): (3019.863101323577, 72.00526315789475), ('Trondheim', 'Oslo', 'avion'): (1124.79009033914, 68.64521276595744), ('Oslo', 'Trondheim', 'avion'): (1166.3191420181638, 73.35514184397162), ('Amsterdam', 'Londres', 'avion'): (1570.3234874094119, 56.29036036036036), ('Londres', 'Amsterdam', 'avion'): (1611.5808663578828, 50.44108108108108), ('Barcelone', 'Palma de Majorque', 'avion'): (1875.0021478076717, 45.485358796296296), ('Bergen', 'Oslo', 'avion'): (822.5813668700935, 48.92169312169312), ('Palma de Majorque', 'Barcelone', 'avion'): (1953.1258548627366, 43.42555168408

In [42]:
df_test = df_planes_arcs[(df_planes_arcs["Ville_DEP"] == "Istanbul") & (df_planes_arcs["Ville_DES"] == "Izmir")]
print(df_test)

      ADEP  ADES  mean_emissions  mean_time  num_flights Ville_DEP Ville_DES
1652  LTFJ  LTBJ     2618.039751  59.928616          742  Istanbul     Izmir
1745  LTFM  LTBJ     3495.562458  69.869721          513  Istanbul     Izmir


In [37]:
Arcs = train_arcs | plane_arcs
print(Arcs)

{('Bruxelles', 'Amsterdam', 'train'): (5.56875, 110), ('Bruxelles', 'Anvers', 'train'): (1.5866666666666667, 34), ('Anvers', 'Rotterdam', 'train'): (1.8225, 36), ('Bruxelles', 'Liège', 'train'): (3.033333333333333, 65), ('Liège', 'Cologne', 'train'): (5.377083333333334, 89), ('Lille', 'Bruxelles', 'train'): (1.1484375, 35), ('Lyon', 'Marseille', 'train'): (0.875, 100), ('Madrid', 'Barcelone', 'train'): (2.964, 156), ('Madrid', 'Seville', 'train'): (2.9069999999999996, 153), ('Seville', 'Cadix', 'train'): (1.596, 84), ('Madrid ', 'Valence', 'train'): (2.318, 122), ('Madrid', 'Malaga', 'train'): (3.192, 168), ('Madrid', 'Saint-Jacques-de-Compostelle', 'train'): (3.439, 181), ('Madrid', 'La Corogne', 'train'): (4.1610000000000005, 219), ('Saint-Jacques-de-Compostelle', 'Vigo', 'train'): (1.007, 53), ('Cordoba', 'Malaga', 'train'): (1.083, 57), ('Barcelone', 'Perpignan', 'train'): (1.1616666666666666, 82), ('Barcelone', 'Saragosse', 'train'): (1.653, 87), ('Madrid', 'Saragosse', 'train'): 

In [ ]:
lines = train_lines['ville_dep'] + '_' + train_lines['ville_arr']
lines = lines.tolist()
for i in range(10):
    print(lines[i])

Bruxelles_Amsterdam
Bruxelles_Anvers
Anvers_Rotterdam
Bruxelles_Liège
Liège_Cologne
Lille_Bruxelles
Lyon_Marseille
Madrid_Barcelone
Madrid_Seville
Seville_Cadix


## Définition des variables

In [ ]:
cities = train_cities['city'].tolist()
train_emissions = []
plane_emissions = []
#dictionnaire emissions de type emissions[ligne][mode] = emission
emissions = {"Paris_Strasbourg": {"plane": 100, "train": 20}}

train_lines = train_lines['ville_dep'] + '_' + train_lines['ville_arr']
train_lines = train_lines.tolist()

modes = ['plane', 'train']

routes = [(line, mode) for line in lines for mode in modes]

# Variable binaire x qui vaut 1 si la ligne i est choisie, 0 sinon
x = LpVariable.dicts(
    "lines", (lines, modes), lowBound=0, upBound=1, cat=LpInteger
)
# Variable binaire qui vaut 1 si on change de mode de transport dans la ville i, 0 sinon
y = LpVariable.dicts(
    "transfert", cities, lowBound=0, upBound=1, cat=LpInteger
)

train_time = train_lines['duree_min'].tolist()
plane_time = plane_lines['time'].tolist()
time = {"Paris_Strasbourg": {"plane": 1, "train": 2}}

## Définition de la fonction objectif

In [ ]:
prob = LpProblem("Flight_emissions", LpMinimize)
prob += (lpSum(emissions[i][m] * x[i][m] for (i,m) in routes), "Total_Emissions")

## Définition des contraintes

In [ ]:
import numpy as np
K = 5
lb = 0.5

for line in lines :
    t_plane = plane_lines['plane_time']
    t_max = t_plane*(1+ k*exp(-lb*t_plane))
    prob += lpSum(time[line][m]*x[line][m])